#  Sentiment Analysis with TensorFlow

A Convolutional Neural Net (CNN) is sometimes used in text classification tasks such as sentiment analysis.  We'll use a CNN built with TensorFlow to perform sentiment analysis in Amazon SageMaker on the IMDB dataset, which consists of movie reviews labeled as having positive or negative sentiment. Three aspects of Amazon SageMaker will be demonstrated:

- How to use Script Mode with a prebuilt TensorFlow container, along with a training script similar to one you would use outside SageMaker. 
- Local Mode training, which allows you to test your code on your notebook instance before creating a full scale training job.
- Batch Transform for offline, asynchronous predictions on large batches of data. 

#  Prepare Dataset

We'll begin by loading the reviews dataset, and padding the reviews so all reviews have the same length.  Each review is represented as an array of numbers, where each number represents an indexed word.  Training data for both Local Mode and Hosted Training must be saved as files, so we'll also save the transformed data to files.

In [4]:
import os
os.system("aws s3 cp s3://sagemaker-workshop-pdx/sentiment-analysis-module/sentiment.py sentiment.py")
os.system("aws s3 cp s3://sagemaker-workshop-pdx/sentiment-analysis-module/setup.sh setup.sh")
os.system("aws s3 cp s3://sagemaker-workshop-pdx/sentiment-analysis-module/daemon.json daemon.json")
from keras.preprocessing import sequence
from keras.datasets import imdb
import numpy as np

max_features = 20000
maxlen = 400



# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test)= imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

25000 train sequences
25000 test sequences
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [5]:
import os

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(os.getcwd(), 'data/csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

In [6]:
import numpy as np

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

# Local Mode Training

Amazon SageMaker’s Local Mode training feature is a convenient way to make sure your code is working as expected before moving on to full scale, hosted training. With Local Mode, you can run quick tests with just a sample of training data, and/or a small number of epochs (passes over the full training set), while avoiding the time and expense of attempting full scale hosted training using possibly buggy code.  

To train in Local Mode, it is necessary to have docker-compose or nvidia-docker-compose (for GPU) installed in the notebook instance. Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

In [7]:
!/bin/bash ./setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


The next step is to set up a TensorFlow Estimator for Local Mode training. A key parameters for the Estimator is the `train_instance_type`, which is the kind of hardware on which training will run. In the case of Local Mode, we simply set this parameter to `local_gpu` to invoke Local Mode training on the GPU, or to `local` if the instance has a CPU. Other parameters of note are the algorithm’s hyperparameters, which are passed in as a dictionary, and a Boolean parameter indicating that we are using Script Mode.

In [8]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

model_dir = '/opt/ml/model'
train_instance_type = 'local'
hyperparameters = {'epochs': 1, 'batch_size': 128}
local_estimator = TensorFlow(entry_point='sentiment.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-keras-sentiment',
                       framework_version='1.12.0',
                       py_version='py3',
                       script_mode=True)

Now we'll briefly train the model in Local Mode.  Since this is just to make sure the code is working, we'll train for only one epoch.  (Note that on a CPU-based notebook instance, this one epoch will take at least 3 or 4 minutes.)  As you'll see from the logs below the cell when training is complete, even when trained for only one epoch, the accuracy of the model on training data is already at almost 80%.  

In [9]:
inputs = {'train': f'file://{train_dir}',
          'test': f'file://{test_dir}'}

local_estimator.fit(inputs)

Creating tmp5owmx06a_algo-1-gg43v_1 ... 
Attaching to tmp5owmx06a_algo-1-gg43v_12mdone
algo-1-gg43v_1  | 2019-11-03 21:04:15,276 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-gg43v_1  | 2019-11-03 21:04:15,283 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-gg43v_1  | 2019-11-03 21:04:15,411 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-gg43v_1  | 2019-11-03 21:04:15,429 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-gg43v_1  | 2019-11-03 21:04:15,442 sagemaker-containers INFO     Invoking user script
algo-1-gg43v_1  | 
algo-1-gg43v_1  | Training Env:
algo-1-gg43v_1  | 
algo-1-gg43v_1  | {
algo-1-gg43v_1  |     "additional_framework_parameters": {},
algo-1-gg43v_1  |     "channel_input_dirs": {
algo-1-gg43v_1  |         "train": "/opt/ml/input/data/train",
algo-1-gg43v_1  |         "test": "/opt/ml/input/data/test"
algo-1-gg43v

#  Hosted Training

After we've confirmed our code seems to be working using Local Mode training, we can move on to use SageMaker's hosted training, which uses compute resources separate from your notebook instance.  Hosted training spins up one or more instances (cluster) for training, and then tears the cluster down when training is complete. In general, hosted training is preferred for doing actual training, especially for large-scale, distributed training. Before starting hosted training, the data must be uploaded to S3. 

In [10]:
s3_prefix = 'tf-keras-sentiment'

traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

train_s3 = sagemaker.Session().upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sagemaker.Session().upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}
print(inputs)

{'train': 's3://sagemaker-us-east-1-340280328827/tf-keras-sentiment/data/train', 'test': 's3://sagemaker-us-east-1-340280328827/tf-keras-sentiment/data/test'}


With the training data now in S3, we're ready to set up an Estimator object for hosted training. It is similar to the Local Mode Estimator, except the `train_instance_type` has been set to a ML instance type instead of a local type for Local Mode. Additionally, we've set the number of epochs to a number greater than one for actual training, as opposed to just testing the code.

In [11]:
train_instance_type = 'ml.p3.2xlarge'
hyperparameters = {'epochs': 10, 'batch_size': 128}

estimator = TensorFlow(entry_point='sentiment.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-keras-sentiment',
                       framework_version='1.12.0',
                       py_version='py3',
                       script_mode=True)

With the change in training instance type and increase in epochs, we simply call `fit` to start the actual hosted training.  At the end of hosted training, you'll see from the logs below the cell that accuracy on the training set has greatly increased, and accuracy on the validation set is around 90%.  The model may be overfitting now (less able to generalize to data it has not yet seen), even though we are employing dropout as a regularization technique.  In a production situation, further investigation would be necessary.

In [12]:
estimator.fit(inputs)

2019-11-03 21:09:53 Starting - Starting the training job...
2019-11-03 21:09:54 Starting - Launching requested ML instances......
2019-11-03 21:10:58 Starting - Preparing the instances for training...
2019-11-03 21:11:47 Downloading - Downloading input data...
2019-11-03 21:12:01 Training - Downloading the training image...
2019-11-03 21:12:36 Training - Training image download completed. Training in progress.2019-11-03 21:12:39,541 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-11-03 21:12:39,937 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch_size": 128,
        "model_dir": "/

#015  128/25000 [..............................] - ETA: 2s - loss: 0.0037 - acc: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  640/25000 [..............................] - ETA: 2s - loss: 0.0083 - acc: 0.9984#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 1152/25000 [>.............................] - ETA: 2s - loss: 0.0061 - acc: 0.9991#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01

#015  128/25000 [..............................] - ETA: 2s - loss: 0.0025 - acc: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  640/25000 [..............................] - ETA: 2s - loss: 0.0012 - acc: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 1152/25000 [>.............................] - ETA: 2s - loss: 9.9993e-04 - acc: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01


2019-11-03 21:13:42 Uploading - Uploading generated training model
2019-11-03 21:13:42 Completed - Training job completed
Training seconds: 115
Billable seconds: 115


# Batch Prediction


If our use case requires individual predictions in near real-time, SageMaker hosted endpoints can be created. Hosted endpoints also can be used for pseudo-batch prediction, but the process is more involved than simply using SageMaker's Batch Transform feature, which is designed for large-scale, asynchronous batch inference.

To use Batch Transform, first we must upload to Amazon S3 some test data in CSV format to be transformed.

In [13]:
csvtestdata_s3_prefix = '{}/data/csv-test'.format(s3_prefix)
csvtest_s3 = sagemaker.Session().upload_data(path='./data/csv-test/', key_prefix=csvtestdata_s3_prefix)
print(csvtest_s3)

s3://sagemaker-us-east-1-340280328827/tf-keras-sentiment/data/csv-test


A Transformer object must be set up to describe the Batch Transform job, including the amount and type of inference hardware to be used.  Then the actual transform job itself is started with a call to the `transform` method of the Transformer.

In [14]:
transformer = estimator.transformer(instance_count=1, instance_type='ml.m5.xlarge')
transformer.transform(csvtest_s3, content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()

Waiting for transform job: tf-keras-sentiment-2019-11-03-21-09-53--2019-11-03-22-24-53-495
..................
INFO:__main__:starting services
INFO:__main__:using default model name: model
INFO:__main__:tensorflow serving model config: 
model_config_list: {
  config: {
    name: "model",
    base_path: "/opt/ml/model",
    model_platform: "tensorflow"
  }
}


INFO:__main__:nginx config: 
load_module modules/ngx_http_js_module.so;

worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr info;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/json;
  access_log /dev/stdout combined;
  js_include tensorflow-serving.js;

  upstream tfs_upstream {
    server localhost:10001;
  }

  upstream gunicorn_upstream {
    server unix:/tmp/gunicorn.sock fail_timeout=1;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;
    client_body_buffer_size 100m;
    subrequest_output_bu

We can now download the batch predictions from S3 to the local filesystem on the notebook instance; the predictions are contained in a file with a .out extension, and are embedded in JSON.  Next we'll load the JSON and examine the predictions, which are confidence scores from 0.0 to 1.0 where numbers close to 1.0 indicate positive sentiment, while numbers close to 0.0 indicate negative sentiment.

In [15]:
import json

batch_output = transformer.output_path
!mkdir -p batch_data/output
!aws s3 cp --recursive $batch_output/ batch_data/output/

with open('batch_data/output/csv-test.csv.out', 'r') as f:
    jstr = json.load(f)
    results = [float('%.3f'%(item)) for sublist in jstr['predictions'] for item in sublist]
    print(results)

download: s3://sagemaker-us-east-1-340280328827/tf-keras-sentiment-2019-11-03-21-09-53--2019-11-03-22-24-53-495/csv-test.csv.out to batch_data/output/csv-test.csv.out
[0.0, 1.0, 0.998, 0.968, 1.0, 0.996, 1.0, 0.0, 1.0, 0.999, 1.0, 0.0, 0.0, 0.663, 1.0, 0.0, 1.0, 0.695, 0.0, 0.0, 1.0, 1.0, 0.92, 1.0, 1.0, 1.0, 0.003, 1.0, 1.0, 0.0, 1.0, 0.317, 0.853, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.993, 0.0, 0.0, 0.0, 0.999, 0.0, 0.003, 1.0, 1.0, 1.0, 0.179, 0.714, 0.99, 0.0, 0.0, 0.002, 0.0, 0.996, 0.0, 0.0, 1.0, 0.102, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.026, 0.0, 1.0, 0.995, 0.0, 0.46, 0.002, 0.999, 1.0, 0.0, 0.0, 0.0, 0.994, 0.0, 0.993, 1.0, 0.998, 0.0, 0.703, 1.0, 0.0, 0.998, 1.0, 0.0, 0.0]


Now let's look at the text of some actual reviews to see the predictions in action.  First, we have to convert the integers representing the words back to the words themselves by using a reversed dictionary.  Next we can decode the reviews, taking into account that the first 3 indices were reserved for "padding", "start of sequence", and "unknown", and removing a string of unknown tokens from the start of the review.

In [21]:
import re

regex = re.compile(r'^[\?\s]+')

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_test[1]])
regex.sub('', first_decoded_review)

"this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances ? the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere ? with sexual tension and psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the ? moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual ? and desperation be patient ? up the atmosphere and pay attention

Overall, this review looks fairly negative.  Let's compare the actual label with the prediction:

In [22]:
def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

print('Labeled sentiment for this review is {}, predicted sentiment is {}'.format(get_sentiment(y_test[3]), 
                                                                                  get_sentiment(results[3])))

Labeled sentiment for this review is negative, predicted sentiment is positive


Our negative sentiment prediction agrees with the label for this review.  Let's now examine another review:

In [19]:
second_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_test[10]])
regex.sub('', second_decoded_review)

"inspired by hitchcock's strangers on a train concept of two men ? murders in exchange for getting rid of the two people messing up their lives throw ? from the train is an original and very inventive comedy take on the idea it's a credit to danny ? that he both wrote and starred in this minor comedy gem br br anne ? is the mother who ? the film's title and it's understandable why she gets under the skin of danny ? with her sharp tongue and relentlessly putting him down for any minor ? billy crystal is the writer who's wife has stolen his book idea and is now being ? as a great new author even appearing on the oprah show to in ? he should be enjoying thus ? gets the idea of ? murders to rid themselves of these ? factors br br of course everything and anything can happen when writer carl ? lets his imagination ? with ? ideas for how the plot develops and it's amusing all the way through providing plenty of laughs and chuckles along the way as well as a good deal of suspense br br for ? 

In [20]:
print('Labeled sentiment for this review is {}, predicted sentiment is {}'.format(get_sentiment(y_test[10]), 
                                                                                  get_sentiment(results[10])))

Labeled sentiment for this review is positive, predicted sentiment is positive


Again, the prediction agreed with the label for the test data.  Note that there is no need to clean up any Batch Transform resources:  after the transform job is complete, the cluster used to make inferences is torn down.

Now that we've reviewed some sample predictions as a sanity check, we're finished.  Of course, in a typical production situation, the data science project lifecycle is iterative, with repeated cycles of refining the model using a tool such as Amazon SageMaker's Automatic Model Tuning feature, and gathering more data.  